In [1]:
# 코드 10-1 원-핫 인코딩 적용
import pandas as pd

class2 = pd.read_csv("../chap10/data/class2.csv")  # 데이터셋을 메모리로 로딩

from sklearn import preprocessing

label_encoder = (
    preprocessing.LabelEncoder()
)  # 데이터를 인코딩하는 데 사용하며, 다음의 OneHotEncoder()와 함께 사용
onehot_encoder = preprocessing.OneHotEncoder()  # 데이터를 숫자 형식으로 표현

train_x = label_encoder.fit_transform(class2["class2"])
train_x

array([2, 2, 1, 0, 1, 0])

In [2]:
# 코드 10-2 코퍼스에 카운터 벡터 적용
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    "This is last chance.",
    "and if you do not have this chance.",
    "you will never get any chance.",
    "will you do get this one?",
    "please, get this chance",
]
vect = CountVectorizer()
vect.fit(corpus)
vect.vocabulary_

{'this': 13,
 'is': 7,
 'last': 8,
 'chance': 2,
 'and': 0,
 'if': 6,
 'you': 15,
 'do': 3,
 'not': 10,
 'have': 5,
 'will': 14,
 'never': 9,
 'get': 4,
 'any': 1,
 'one': 11,
 'please': 12}

In [3]:
# 코드 10-3 배열 변환
vect.transform(["you will never get any chance."]).toarray()

array([[0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]])

In [4]:
# 코드 10-4 불용어를 제거한 카운터 벡터
vect = CountVectorizer(stop_words=["and", "is", "please", "this"]).fit(
    corpus
)  # stop_words를 사용하여 is, an 같은 불용어 제거
vect.vocabulary_

{'last': 6,
 'chance': 1,
 'if': 5,
 'you': 11,
 'do': 2,
 'not': 8,
 'have': 4,
 'will': 10,
 'never': 7,
 'get': 3,
 'any': 0,
 'one': 9}

In [5]:
# 코드 10-5 TF-IDF를 적용한 후 행렬로 표현
from sklearn.feature_extraction.text import TfidfVectorizer

doc = ["I like machine learning", "I love deep learning", "I run everyday"]
tfidf_vectorizer = TfidfVectorizer(min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(doc)
doc_distance = tfidf_matrix * tfidf_matrix.T
print(
    "유사도를 위한",
    str(doc_distance.get_shape()[0]),
    "x",
    str(doc_distance.get_shape()[1]),
    "행렬을 만들었습니다.",
)
print(doc_distance.toarray())

유사도를 위한 3 x 3 행렬을 만들었습니다.
[[1.       0.224325 0.      ]
 [0.224325 1.       0.      ]
 [0.       0.       1.      ]]


In [9]:
%pip install nltk gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 47.3 MB/s eta 0:00:0000:0100:01
  Using cached smart_open-7.0.5-py3-none-any.whl (61 kB)

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
# 코드 10-6 데이터셋을 메모리로 로딩하고 토큰화 적용
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings

warnings.filterwarnings(action="ignore")
import gensim
from gensim.models import Word2Vec

sample = open("../chap10/data/peter.txt", "r", encoding="UTF8")  # 피터팬 데이터셋 로딩
s = sample.read()

f = s.replace("\n", " ")  # 줄바꿈(\n)을 “ ”(공백)으로 변환
data = []

for i in sent_tokenize(f):  # 로딩한 파일의 각 문장마다 반복
    temp = []
    for j in word_tokenize(i):  # 문장을 단어로 토큰화
        temp.append(j.lower())  # 토큰화된 단어를 소문자로 변환하여 temp에 저장
    data.append(temp)

data

[['once',
  'upon',
  'a',
  'time',
  'in',
  'london',
  ',',
  'the',
  'darlings',
  'went',
  'out',
  'to',
  'a',
  'dinner',
  'party',
  'leaving',
  'their',
  'three',
  'children',
  'wendy',
  ',',
  'jhon',
  ',',
  'and',
  'michael',
  'at',
  'home',
  '.'],
 ['after',
  'wendy',
  'had',
  'tucked',
  'her',
  'younger',
  'brothers',
  'jhon',
  'and',
  'michael',
  'to',
  'bed',
  ',',
  'she',
  'went',
  'to',
  'read',
  'a',
  'book',
  '.'],
 ['she', 'heard', 'a', 'boy', 'sobbing', 'outside', 'her', 'window', '.'],
 ['he', 'was', 'flying', '.'],
 ['there', 'was', 'little', 'fairy', 'fluttering', 'around', 'him', '.'],
 ['wendy', 'opened', 'the', 'window', 'to', 'talk', 'to', 'him', '.'],
 ['“', 'hello', '!'],
 ['who', 'are', 'you', '?'],
 ['why', 'are', 'you', 'crying', '”', ',', 'wendy', 'asked', 'him', '.'],
 ['“', 'my', 'name', 'is', 'peter', 'pan', '.'],
 ['my',
  'shadow',
  'wouldn',
  '’',
  't',
  'stock',
  'to',
  'me.',
  '”',
  ',',
  'he',
  'rep

In [12]:
# 코드 10-7 데이터셋에 CBOW 적용 후 ‘peter’와 ‘wendy’의 유사성 확인
model1 = gensim.models.Word2Vec(data, min_count=1, vector_size=100, window=5, sg=0)  # ①

print(
    "Cosine similarity between 'peter' " + "'wendy' - CBOW : ",
    model1.wv.similarity("peter", "wendy"),
)  # 결과 출력

Cosine similarity between 'peter' 'wendy' - CBOW :  0.074393824


In [13]:
# 코드 10-8 ‘peter’와 ‘hook’의 유사성 확인
print(
    "Cosine similarity between 'peter' " + "'hook' - CBOW : ",
    model1.wv.similarity("peter", "hook"),
)

Cosine similarity between 'peter' 'hook' - CBOW :  0.027709836


In [14]:
# 코드 10-9 데이터셋에 skip-gram 적용 후 ‘peter’와 ‘wendy’의 유사성 확인
model2 = gensim.models.Word2Vec(
    data, min_count=1, vector_size=100, window=5, sg=1
)  # skip-gram 모델 사용

print(
    "Cosine similarity between 'peter' " + "'wendy' - Skip Gram : ",
    model2.wv.similarity("peter", "wendy"),
)  # 결과 출력

Cosine similarity between 'peter' 'wendy' - Skip Gram :  0.40088683


In [15]:
# 코드 10-10 ‘peter’와 ‘hook’의 유사성
print(
    "Cosine similarity between 'peter' " + "'hook' - Skip Gram : ",
    model2.wv.similarity("peter", "hook"),
)

Cosine similarity between 'peter' 'hook' - Skip Gram :  0.5201673


In [16]:
# 코드 10-11 라이브러리 및 데이터 호출
from gensim.test.utils import common_texts
from gensim.models import FastText

model = FastText(
    "../chap10/data/peter.txt", vector_size=4, window=3, min_count=1, epochs=10
)  # ①

In [17]:
# 코드 10-12 ‘peter’, ‘wendy’에 대한 코사인 유사도
sim_score = model.wv.similarity("peter", "wendy")
print(sim_score)

0.4592452


In [18]:
# 코드 10-13 ‘peter’와 ‘hook’에 대한 코사인 유사도
sim_score = model.wv.similarity("peter", "hook")
print(sim_score)

0.043825686


In [21]:
# 코드 10-14 라이브러리와 사전 훈련된 모델 호출
from __future__ import print_function
from gensim.models import (
    KeyedVectors,
)  # gensim은 자연어를 벡터로 변환하는 데 필요한 편의 기능을 제공하는 라이브러리입니다.

model_kr = KeyedVectors.load_word2vec_format(
    "../chap10/data/wiki.ko.vec"
)  # wiki.ko.vec 파일을 메모리로 불러옵니다.

In [22]:
# 코드 10-15 ‘노력’과 유사한 단어와 유사도 확인
find_similar_to = "노력"

for similar_word in model_kr.similar_by_word(find_similar_to):
    print("Word: {0}, Similarity: {1:.2f}".format(similar_word[0], similar_word[1]))

Word: 노력함, Similarity: 0.80
Word: 노력중, Similarity: 0.75
Word: 노력만, Similarity: 0.72
Word: 노력과, Similarity: 0.71
Word: 노력의, Similarity: 0.69
Word: 노력가, Similarity: 0.69
Word: 노력이나, Similarity: 0.69
Word: 노력없이, Similarity: 0.68
Word: 노력맨, Similarity: 0.68
Word: 노력보다는, Similarity: 0.68


In [25]:
# 코드 10-17 라이브러리 호출 및 데이터셋 로딩
import numpy as np

%matplotlib notebook
import matplotlib.pyplot as plt

plt.style.use("ggplot")
from sklearn.decomposition import PCA
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

# 아래 상대 경로에 오류가 발생한다면 절대 경로로 지정해 주세요.
glove_file = "../chap10/data/glove.6B.100d.txt"  # ①
word2vec_glove_file = get_tmpfile(
    "glove.6B.100d.word2vec.txt"
)  # 글로브 데이터를 워드투벡터 형태로 변환
glove2word2vec(glove_file, word2vec_glove_file)  # ②

(400000, 100)

In [26]:
# 코드 10-18 ‘bill’과 유사한 단어의 리스트를 반환
model = KeyedVectors.load_word2vec_format(
    word2vec_glove_file
)  # load_word2vec_format() 메서드를 이용하여 word2vec.c 형식으로 벡터를 가져옵니다.
model.most_similar(
    "bill"
)  # 단어(bill) 기준으로 가장 유사한 단어들의 리스트를 보여 줍니다.

[('legislation', 0.8072139620780945),
 ('proposal', 0.730686366558075),
 ('senate', 0.7142541408538818),
 ('bills', 0.704440176486969),
 ('measure', 0.6958035230636597),
 ('passed', 0.690624475479126),
 ('amendment', 0.6846879720687866),
 ('provision', 0.6845567226409912),
 ('plan', 0.6816462874412537),
 ('clinton', 0.6663140058517456)]

In [27]:
# 코드 10-19 ‘cherry’와 유사한 단어의 리스트를 반환
model.most_similar(
    "cherry"
)  # 단어(cherry) 기준으로 가장 유사한 단어들의 리스트를 보여 줍니다.

[('peach', 0.688809871673584),
 ('mango', 0.683819055557251),
 ('plum', 0.6684104204177856),
 ('berry', 0.6590359210968018),
 ('grove', 0.658155083656311),
 ('blossom', 0.6503506302833557),
 ('raspberry', 0.6477391719818115),
 ('strawberry', 0.6442098021507263),
 ('pine', 0.6390928626060486),
 ('almond', 0.6379212737083435)]

In [28]:
# 코드 10-20 ‘cherry’와 관련성이 없는 단어의 리스트를 반환
model.most_similar(negative=["cherry"])  # 단어(cherry)와 관련성이 없는 단어들을 반환

[('kazushige', 0.48343509435653687),
 ('askerov', 0.4778185784816742),
 ('lakpa', 0.46915265917778015),
 ('ex-gay', 0.4571332633495331),
 ('tadayoshi', 0.4522107243537903),
 ('turani', 0.44810065627098083),
 ('saglam', 0.4469599425792694),
 ('aijun', 0.4435270130634308),
 ('adjustors', 0.44235292077064514),
 ('nyum', 0.4423118233680725)]

In [29]:
# 코드 10-21 ‘woman’, ‘king’과 유사성이 높으면서 ‘man’과 관련성이 없는 단어를 반환
result = model.most_similar(
    positive=["woman", "king"], negative=["man"]
)  # woman, king과 유사성이 높으면서 man과 관련성이 없는 단어를 반환
print("{}: {:.4f}".format(*result[0]))

queen: 0.7699


In [30]:
# 코드 10-22 ‘australia’, ‘beer’, ‘france’와 관련성이 있는 단어를 반환
def analogy(x1, x2, y1):
    result = model.most_similar(positive=[y1, x2], negative=[x1])
    return result[0][0]


analogy("australia", "beer", "france")

'champagne'

In [31]:
# 코드 10-23 ‘tall’, ‘tallest’, ‘long’ 단어를 기반으로 새로운 단어를 유추
analogy("tall", "tallest", "long")

'longest'

In [32]:
# 코드 10-24 ‘breakfast cereal dinner lunch’ 중 유사도가 낮은 단어를 반환
print(
    model.doesnt_match("breakfast cereal dinner lunch".split())
)  # 유사도가 가장 낮은 단어를 반환

cereal
